In [1]:
#!pip install "tensorflow==2.14.0"
#!pip install hexdump

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
print(f"TensorFlow {tf.__version__} devices:\n{tf.config.list_physical_devices()}")

TensorFlow 2.14.0 devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
### ignore TensorFlow INFO messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [5]:
model = tf.keras.Sequential()
model.add(layers.Dense(8, input_dim=2, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))

In [6]:
model.compile(
  loss='binary_crossentropy', 
  optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1), 
  metrics=['accuracy'])

In [7]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [8]:
epochs = 500
model.fit(training_data, target_data, epochs=epochs)

Epoch 1/500
1/1 [==============================] - 0s 138ms/step - loss: 0.7555 - accuracy: 0.5000
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6997 - accuracy: 0.7500
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6983 - accuracy: 0.5000
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6978 - accuracy: 0.5000
Epoch 5/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6973 - accuracy: 0.5000
Epoch 6/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6968 - accuracy: 0.5000
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6964 - accuracy: 0.5000
Epoch 8/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6959 - accuracy: 0.5000
Epoch 9/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6955 - accuracy: 0.5000
Epoch 10/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6951 - accuracy: 0.5000
Epoch 11/500
1/1 [=========

1/1 [==============================] - 0s 5ms/step - loss: 0.0596 - accuracy: 1.0000
Epoch 496/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0592 - accuracy: 1.0000
Epoch 497/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0588 - accuracy: 1.0000
Epoch 498/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0584 - accuracy: 1.0000
Epoch 499/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0580 - accuracy: 1.0000
Epoch 500/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0576 - accuracy: 1.0000


In [9]:
model.predict(training_data)

1/1 [==============================] - 0s 51ms/step


array([[0.03989676],
       [0.9591703 ],
       [0.9634982 ],
       [0.04992343]], dtype=float32)

## C-Code

In [10]:
### tinymlgen - see https://github.com/eloquentarduino/tinymlgen

import re
import hexdump
import tensorflow as tf


def port(model, optimize=False, variable_name='model_data', pretty_print=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimize:
        if isinstance(optimize, bool):
            optimizers = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
        else:
            optimizers = optimize
        converter.optimizations = optimizers
    tflite_model = converter.convert()
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    preamble = '''
// if having troubles with min/max, uncomment the following
// #undef min    
// #undef max

#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

'''
    return preamble + c

In [11]:
c_code = port(model, optimize=[tf.lite.Optimize.DEFAULT], pretty_print = True)

path = os.getcwd() + "/../pico-tflmicro/examples/xor/"
open(path + "model.cpp", "w").write('#include "model.h"\n' + c_code)

INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmpz8xjswpq/assets


INFO:tensorflow:Assets written to: /var/folders/v9/t6g_nwb51r386g11xp6_3q0w0000gn/T/tmpz8xjswpq/assets
2023-10-21 15:41:10.473820: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-10-21 15:41:10.473834: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.


12179